In [52]:
import pandas as pd
from sqlalchemy import create_engine
import json
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")
import plotly.express as px
import numpy as np


In [2]:
# Read the MySQL configuration from the JSON file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

In [3]:
# Extract MySQL connection details
mysql_config = config.get('mysql', {})
username = mysql_config.get('username', 'default_username')
password = mysql_config.get('password', 'default_password')
host = mysql_config.get('host', 'localhost')
database_name = mysql_config.get('database_name', 'your_database')
table_name = 'measurments_rewa'

In [4]:
# Create the MySQL database connection string
db_url = f"mysql+mysqlconnector://{username}:{password}@{host}/{database_name}"

# Create an SQLAlchemy engine
engine = create_engine(db_url)

# Use the engine to connect to the database
connection = engine.connect()

In [5]:
# Specify the SQL query to retrieve data from a table
query = "SELECT * FROM measurments_rewa"

# Use Pandas to read data from the database into a DataFrame
df = pd.read_sql(query, connection)

# Close the database connection
connection.close()

In [58]:
# Assuming 'df' is your DataFrame with wind data
# You can define your bins and labels for wind speed
bins = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26]  # Customize bin edges as needed
labels = ['0-2', '2-4', '4-6', '6-8', '8-10', '10-12', '12-14', '14-16', '16-18', '18-20', '20-22', '22-24','24-26']

# Create a new column 'WindSpeedBin' with the binned wind speeds
df['WindSpeedBin'] = pd.cut(df['WindSpeed'], bins=bins, labels=labels)

# Define wind direction bins
bins = [0, 45, 90, 135, 180, 225, 270, 315, 360]
labels = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']

# Bin wind directions and calculate mean wind speed for each bin
df['WindDirBin'] = pd.cut(df['WindDir'], bins=bins, labels=labels)
#aggregated_data = df.groupby('WindDirBin')['WindSpeed'].count().reset_index()
aggregated_data = df.groupby(['WindDirBin', 'WindSpeedBin']).agg({'WindSpeed':'mean','WindDir':'count'}).reset_index()

In [59]:
aggregated_data

,WindDirBin,WindSpeedBin,WindSpeed,WindDir
0,N,0-2,1.432031,768
1,N,2-4,3.116319,1581
2,N,4-6,4.998579,2041
3,N,6-8,6.940764,1413
4,N,8-10,8.944770,1023
...,...,...,...,...
99,NW,16-18,16.990625,224
100,NW,18-20,18.975325,77
101,NW,20-22,20.954762,42
102,NW,22-24,22.783333,18


In [60]:
aggregated_data.rename({'WindDir':'Count'}, inplace=True)
aggregated_data['Frequency'] = np.round(aggregated_data['WindDir']/aggregated_data['WindDir'].sum(),2)*100

In [61]:
aggregated_data

,WindDirBin,WindSpeedBin,WindSpeed,WindDir,Frequency
0,N,0-2,1.432031,768,1.0
1,N,2-4,3.116319,1581,1.0
2,N,4-6,4.998579,2041,2.0
3,N,6-8,6.940764,1413,1.0
4,N,8-10,8.944770,1023,1.0
...,...,...,...,...,...
99,NW,16-18,16.990625,224,0.0
100,NW,18-20,18.975325,77,0.0
101,NW,20-22,20.954762,42,0.0
102,NW,22-24,22.783333,18,0.0


In [64]:
# Create a wind rose plot
fig = px.bar_polar(aggregated_data, r='Frequency', theta='WindDirBin', color='WindSpeed',
                   template='plotly_dark', color_discrete_sequence=px.colors.sequential.Plasma_r)
fig.show()